In [ ]:
import nibabel as nib
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv
import glob
import shutil
import seaborn as sns
import sklearn

In [ ]:
from statsmodels.stats.multitest import fdrcorrection
import scipy.stats as stats
from scipy.stats import pearsonr

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.feature_selection import RFE, SelectKBest, mutual_info_classif,f_classif
from sklearn.model_selection import (KFold, train_test_split, cross_validate, GridSearchCV, RepeatedStratifiedKFold,
                                     cross_val_score, GroupKFold, StratifiedGroupKFold, StratifiedKFold)
from sklearn.metrics import (roc_auc_score, recall_score, make_scorer, f1_score, confusion_matrix, roc_curve, accuracy_score,
                             precision_recall_curve, auc, plot_precision_recall_curve)
from sklearn.base import clone
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC,LinearSVC
from xgboost import XGBClassifier
import xgboost
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis                              
from sklearn.tree import DecisionTreeClassifier
from feature_engine.selection import RecursiveFeatureAddition

## Data Preprocessing

### Reading Data

In [ ]:
# initialize the file paths to hold all the images separated by sequence and SRS Date
root = '/Users/cxl037/PycharmProjects/pythonProject1/Example_Data/'
root_ML = '/Users/cxl037/PycharmProjects/DeepLearning'
model_path = os.path.join(root_ML, 'Models', 'Post-Pre')
if not os.path.exists(model_path):
    os.mkdir(model_path)
PreT1Bravo_path = os.path.join(root, 'PreT1Bravo_Immuno')
PreT1Bravo_batchfile = os.path.join(PreT1Bravo_path, 'radiomics_features_re.csv')
PreT1Vasc_path = os.path.join(root, 'PreT1Vasc_Immuno')
PreT1Vasc_batchfile = os.path.join(PreT1Vasc_path, 'radiomics_features_re.csv')
PreT2Flair_path = os.path.join(root, 'PreT2Flair_Immuno')
PreT2Flair_batchfile = os.path.join(PreT2Flair_path, 'radiomics_features_re.csv')

In [ ]:
# Read T1Bravo data and add T1_ prefix to all the feature columnsextractedFeatures = pd.read_csv(PreT1Bravo_batchfile)
extractedFeatures = extractedFeatures.add_prefix('Pre_T1_')
extractedFeatures.rename(columns = {'Pre_T1_Image': 'T1_Image', 'Pre_T1_Mask': 'Mask'}, inplace = True)
# Get patientID for future merging
extractedFeatures['PatientID'] = extractedFeatures.apply(lambda row: row['Mask'][:7], axis=1)
extractedFeatures['PatientID'] = extractedFeatures['PatientID'].astype(str)
extractedFeatures

In [ ]:
# Read T1Vasc data and add Vasc_ prefix to all the feature columns
extractedFeatures2 = pd.read_csv(PreT1Vasc_batchfile)
extractedFeatures2 = extractedFeatures2.add_prefix('Pre_Vasc_')
extractedFeatures2.rename(columns = {'Pre_Vasc_Image': 'Vasc_Image', 'Pre_Vasc_Mask': 'Mask'}, inplace = True)
extractedFeatures2

In [ ]:
# Read T2Flair data and add T2_ prefix to all the feature columns
extractedFeatures3 = pd.read_csv(PreT2Flair_batchfile)
extractedFeatures3 = extractedFeatures3.add_prefix('Pre_T2_')
extractedFeatures3.rename(columns = {'Pre_T2_Image': 'T2_Image', 'Pre_T2_Mask': 'Mask'}, inplace = True)
extractedFeatures3

In [ ]:
# Merge T1Bravo, T1Vasc, and T2Flair columns together
allFeatures = pd.merge(extractedFeatures, extractedFeatures2, on='Mask', how = 'left' )
allFeatures = pd.merge(allFeatures, extractedFeatures3, on='Mask', how = 'left' )
allFeatures

In [ ]:
# Get patient immunotherapy information
patientDetails = pd.read_excel('/Users/cxl037/PycharmProjects/pythonProject1/SRS_immune_list.xlsx')
patientDetails['PatientID'] = patientDetails.apply(lambda row: str(row['MRN']) if len(str(row['MRN'])) == 7 else '0' + str(row['MRN']), axis=1)

In [ ]:
# Use patientID to merge radiomic_features.csv and SRS_immune_list.xlsx
allFeatures = pd.merge(allFeatures, patientDetails, on='PatientID', how = 'left')
allFeatures

### Filtering Data

In [ ]:
# Drop all diagnostic features
keep = ['Immunotherapy_prior_3_months', 'PatientID', 'Mask']
remove_cols = [feature for feature in allFeatures.columns 
               if not (feature.startswith("Pre_T2_original") or feature.startswith("Pre_T1_original") 
               or feature.startswith("Pre_Vasc_original") or feature.startswith("Post_T1_original")
               or feature.startswith("Post_Vasc_original") or feature.startswith("Post_T2_original")
               or feature in keep)]
filteredFeatures = allFeatures.drop(remove_cols, axis = 1)
filteredFeatures = filteredFeatures.dropna() # remove rows with NA values
filteredFeatures

In [ ]:
# Get patient complete response information
cr = pd.read_excel('/Users/cxl037/PycharmProjects/pythonProject1/SRS_immuno_CR.xlsx')
cr['Mask'] = cr.apply(lambda row: row['Mask'].strip(), axis=1)
# display(cr['Mask'])
# display(filteredFeatures['Mask'])

# Use Mask to merge features and SRS_immune_CR.xlsx
filteredFeatures = pd.merge(filteredFeatures, cr, on='Mask', how = 'left')
filteredFeatures = filteredFeatures.dropna()
filteredFeatures['CR'] = filteredFeatures['CR'].astype(int)
filteredFeatures

In [ ]:
# filter out all the small lesions less than a certain diameter
filteredFeatures = filteredFeatures.loc[filteredFeatures['Pre_T1_original_shape_Maximum3DDiameter'] >= 10]
with pd.option_context('display.max_rows', None):  # more options can be specified also
    display(filteredFeatures['Pre_T1_original_shape_Maximum3DDiameter'])
    display(filteredFeatures)

In [ ]:
# Get training data and labels
X = filteredFeatures.drop(['CR', 'PatientID', 'Mask', 'Immunotherapy_prior_3_months'], axis=1)
y = filteredFeatures['CR']
feature_names = list(X.columns)
allSubjects = list(filteredFeatures.loc[:, 'PatientID'])
rd=42
y

### Removing Correlated Features

In [ ]:
from statsmodels.stats.multitest import fdrcorrection
from scipy.stats import pearsonr

def get_correlated_features(df, threshold=0.95, max_pvalue=0.05):
    corr_matrix = np.zeros((df.shape[1], df.shape[1]))
    pvalue_matrix = np.zeros((df.shape[1], df.shape[1]))
    msk_cols = list(df.columns)

    # initializes (i,j) with the correlation coefficient and p-value for testing non-correlation 
    # between columns i and j
    for i in range(df.shape[1]):
        for j in range(df.shape[1]):
            corrtest = pearsonr(df[df.columns[i]], df[df.columns[j]])
            corr_matrix[i, j] = corrtest[0]
            pvalue_matrix[i, j] = corrtest[1]
    
    p_values = []
    for i in range(df.shape[1]):
        for j in range(i + 1, df.shape[1]):
            p_values.append(pvalue_matrix[i, j])
    
    # corrected p-values to make sure that no false significant results occur
    p_values_corrected = fdrcorrection(p_values, alpha=0.05, method='indep', is_sorted=False)[1]
    pvalues_corrected_matrix = np.zeros((df.shape[1], df.shape[1]))
    

    k = 0
    for i in range(df.shape[1]):
        for j in range(i + 1, df.shape[1]):
            pvalues_corrected_matrix[i, j] = p_values_corrected[k]
            pvalues_corrected_matrix[j, i] = p_values_corrected[k]
            k += 1

    to_drop_matrix = np.zeros((df.shape[1], df.shape[1]))
    
    # Create a matrix where (i, j) is correlated but j > i, in other words only consider upper triangular indices
    for i in range(df.shape[1]):
        for j in range(i + 1, df.shape[1]):
            if pvalues_corrected_matrix[i, j] < max_pvalue and abs(corr_matrix[i, j]) > threshold:
                to_drop_matrix[i, j] = 1
            else:
                if abs(corr_matrix[i, j]) > threshold:
                    print(msk_cols[i] + " * " + msk_cols[j] + 'corr, pvalue, fdr: %f, %f, %f' % (
                        np.round(corr_matrix[i, j], decimals=3),
                        np.round(pvalue_matrix[i, j], decimals=3),
                        np.round(pvalues_corrected_matrix[i, j], decimals=3)))
                to_drop_matrix[i, j] = 0

    upper = pd.DataFrame(to_drop_matrix)
    to_drop = [column for column in upper.columns if any(upper[column] == 1)]
    
    correlated_feats = {}
    for feature in msk_cols:
        correlated_feats[feature] = set()
    for i in to_drop:
        for j in upper.columns:
            if upper[i][j] > threshold:
                correlated_feats[msk_cols[j]].add(msk_cols[i]) # adds the dropped features as the value

    feats_to_drop = [msk_cols[i] for i in to_drop]

    # show how the kept features correlate with the dropped features
    new_correlated_feats = {}
    for feat in correlated_feats.keys():
        if feat not in feats_to_drop and len(correlated_feats[feat]) > 0:
            new_correlated_feats[feat] = correlated_feats[feat]
    correlated_feats = new_correlated_feats


    printDict = {'corr_matrix': corr_matrix,
                 'p_values_matrix': pvalue_matrix,
                 'p_values_corrected_matrix': pvalues_corrected_matrix,
                 'correlated_feats': correlated_feats,
                 'feats_to_drop': feats_to_drop
                }
    return printDict
    

result = get_correlated_features(X)


corr_matrix = abs(result['corr_matrix'])
corr_pvalues = result['p_values_matrix']
corr_pvalues_corrected = result['p_values_corrected_matrix']
correlated_feats = result['correlated_feats']
feats_to_drop = result['feats_to_drop']

print('feats_to_drop',feats_to_drop)
print(len(feats_to_drop))
print('\n')
print('correlated_feats',correlated_feats)

In [ ]:
# Drop all correlated features
X.drop(feats_to_drop, axis = 1, inplace=True)
feature_names = list(X.columns)
# fig, ax = plt.subplots(figsize=(30,30))
# ax = sns.heatmap(X.corr())
X

## Analyzing the Data

In [ ]:
# Split data into no complete response and complete response
no_cr = X.loc[y==0]
cr = X.loc[y==1]
display(no_cr)
display(cr)

In [ ]:
# Run t-test on each of the features to see if there is a difference in no CR and CR
import scipy.stats as stats
from statsmodels.stats.multitest import fdrcorrection
significant_features = []
p_values = []
count = 0
for feature in cr.columns:
    t_stat, pvalue = stats.ttest_ind(cr[feature], no_cr[feature], equal_var=True)
    p_values.append(pvalue)
p_values_corrected = fdrcorrection(p_values, alpha=0.05, method='indep', is_sorted=False)[1]
for i in range(len(p_values)):
    if p_values_corrected[i] <= 0.05:
        print(cr.columns[i])
        count += 1
print(count)
# p_values_notsorted = p_values_corrected.copy()
# p_values_corrected.sort(reverse=True)
# for i in p_values_corrected:
#     print(cr.columns[p_values_notsorted.index(i)])
    
# sig_features = []
# for i in range(len(p_values)):
#     if p_values_corrected[i] <= 0.05:
#         sig_features.append(cr.columns[i])
# print(sig_features)

In [ ]:
# Univariate Analysis
sc = StandardScaler()
X_train_ori = X
y_train_ori = y
X_train = sc.fit_transform(X)
X_train = pd.DataFrame(X_train)
X_train.columns = X.columns
X_train

lr = LogisticRegression()
AUCforUniLogit = {}
for (featureName, featureData) in X_train.iteritems():
    lr = lr.fit(featureData.values.reshape(-1, 1),y)
    roc_auc = roc_auc_score(y, lr.predict_proba(featureData.values.reshape(-1, 1))[:, 1])
    AUCforUniLogit[featureName] = roc_auc
AUCforUniLogitTop = dict(sorted(AUCforUniLogit.items(), key=lambda item: item[1], reverse=True))
AUCforUniLogitRank = list(AUCforUniLogitTop.keys())
AUCforUniLogitTop20 = dict(sorted(AUCforUniLogit.items(), key=lambda item: item[1], reverse=True)[:20])
print("{:<50} {:<50}".format('feature','auc value'))
for key, value in AUCforUniLogitTop20.items():
    print("{:<50} {:<50}".format(key, value))

In [ ]:
# import matplotlib.lines as mlines
# feature = 'Pre_T1_original_glszm_SizeZoneNonUniformity'
# df1 = cr[feature]
# df2 = no_cr[feature]
# plt.plot(df1, len(df1) * [1], 'x')
# plt.plot(df2, len(df2) * [2], 'o')
# plt.title('T1_SizeZoneNonUniformity')
# markers = [mlines.Line2D([], [], color='orange', marker='o', markersize=5, label='No CR'),
#           mlines.Line2D([], [], color='blue', marker='x', markersize=5, label='CR')]
# plt.legend(handles=markers, loc='best')
# plt.show()
# t_stat, pvalue = stats.ttest_ind(cr[feature], no_cr[feature], equal_var=True)
# print("t-value: ", t_stat)
# print("pvalue: ", pvalue)
# print('CR mean: ', cr[feature].mean())
# print('No CR mean: ', no_cr[feature].mean())

In [ ]:
# immuno = filteredFeatures.loc[filteredFeatures['Immunotherapy_prior_3_months'] == 1]
# cr_immuno = immuno.loc[immuno['CR'] == 1]
# nocr_immuno = immuno.loc[immuno['CR'] == 0]
# CR_immuno = len(cr_immuno)
# immuno_length = len(immuno)
# volume = immuno['Pre_T1_original_shape_MeshVolume'].sum()
# cr_volume = cr_immuno['Pre_T1_original_shape_MeshVolume'].sum()
# nocr_volume = nocr_immuno['Pre_T1_original_shape_MeshVolume'].sum()
# print("#CR in immuno cohort: ", CR_immuno)
# print('#lesions in immuno cohort: ', immuno_length)
# print('Average volume overall: ', volume/immuno_length)
# print('ratio: ', CR_immuno/immuno_length)
# print('Average volume in CR cohort: ', cr_volume/CR_immuno)
# print('Average volume in not CR cohort: ', nocr_volume/(immuno_length - CR_immuno))

In [ ]:
# no_immuno = filteredFeatures.loc[filteredFeatures['Immunotherapy_prior_3_months'] == 0]
# cr_no_immuno = no_immuno.loc[no_immuno['CR'] == 1]
# nocr_no_immuno = no_immuno.loc[no_immuno['CR'] == 0]
# CR_no = len(cr_no_immuno)
# no_length = len(no_immuno)
# volume = no_immuno['Pre_T1_original_shape_MeshVolume'].sum()
# cr_volume = cr_no_immuno['Pre_T1_original_shape_MeshVolume'].sum()
# nocr_volume = nocr_no_immuno['Pre_T1_original_shape_MeshVolume'].sum()
# print("#CR in no immuno cohort: ", CR_no)
# print('#lesions in no immuno cohort: ', no_length)
# print('Average volume overall: ', volume/no_length)
# print('Average volume in CR cohort: ', cr_volume/CR_no)
# print('Average volume in not CR cohort: ', nocr_volume/(no_length - CR_no))
# print('ratio: ', CR_no/no_length)
# # immuno = filteredFeatures.loc[filteredFeatures['Immunotherapy_prior_3_months'] == 1]
# # immuno = immuno.drop(['Immunotherapy_prior_3_months', 'PatientID'],axis=1)

In [ ]:
# no_cr = filteredFeatures.loc[filteredFeatures['CR']==0]
# cr = filteredFeatures.loc[filteredFeatures['CR']==1]
# with pd.option_context('display.max_rows', None):  # more options can be specified also
#     display(no_cr['Pre_T1_original_shape_Maximum3DDiameter'])
#     display(cr['Pre_T1_original_shape_Maximum3DDiameter'])

## Running Models

### Defining variables and Functions

In [ ]:
# SMOTE
from collections import Counter
import imblearn
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks, NearMiss

In [ ]:
# constants
N_SPLITS = 3
N_REPEATS = 10
cv = RepeatedStratifiedKFold(n_splits=N_SPLITS, n_repeats=N_REPEATS, random_state=rd)

In [ ]:
def cross_val(clf, cv, resample=None):
    train_scores = []
    test_scores = []
    aucpr_scores = []
    for fold, (train_idxs, test_idxs) in enumerate(cv.split(X, y, groups=allSubjects)):
        print("Repeat :", fold // N_SPLITS + 1)
        print("Fold :", fold % N_SPLITS + 1)
#         print("train length:", len(train_idxs))
#         print("test length:", len(test_idxs))
        
        # split data
        X_train, X_test = X.iloc[train_idxs], X.iloc[test_idxs]
        y_train, y_test = y.iloc[train_idxs], y.iloc[test_idxs]
        counter = Counter(y_train)
        print(counter)
        
        # resample if needed
        if resample:
            X_train, y_train = resample.fit_resample(X_train, y_train)
            counter = Counter(y_train)
            print(counter)
            
        # fit model
        clf.fit(X_train, y_train)
        
        # calculate scores
        train_auc_score = roc_auc_score(y_train, clf.predict_proba(X_train)[:, 1])
        print("TRAIN SCORE: ", train_auc_score)
        train_scores.append(train_auc_score)
        
        test_auc_score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        test_scores.append(test_auc_score)
        print("TEST AUC SCORE: ", test_auc_score)
        
        precision, recall, thresholds = precision_recall_curve(y_test, clf.predict_proba(X_test)[:, 1])
        auc_precision_recall = auc(recall, precision)
        aucpr_scores.append(auc_precision_recall)
        print("TEST PR AUC SCORE: ",auc_precision_recall)
        
        #     predictions = rfc.predict_proba(X_test)[:, 1]
        #     actual = list(y_test)
        #     print('Pred: ', predictions)
        #     print('Actual: ', actual) 
        
        # AUC graph
        #     fpr, tpr, _ = roc_curve(y_test, rfc.predict_proba(X_test)[:, 1])
        #     plt.plot(fpr, tpr, marker='.', label='Random Forest')
        #     plt.xlabel('False Positive Rate')
        #     plt.ylabel('True Positive Rate')
        #     # show the legend
        #     plt.legend()
        #     # show the plot
        #     plt.show()
    
    # averages across all fold
    print("")
    print("MEAN TRAIN SCORES:", np.mean(train_scores))
    print("STD TRAIN SCORES:", np.std(train_scores))
    print("MEAN TEST SCORES:", np.mean(test_scores))
    print("STD TEST SCORES:", np.std(test_scores))
    print("MEAN AUCPR TEST SCORES:", np.mean(aucpr_scores))
    print("STD AUCPR TEST SCORES:", np.std(aucpr_scores))

### XGBoost

In [ ]:
xgb = XGBClassifier(objective="binary:logistic", random_state=rd, eval_metric='auc',use_label_encoder=False,
                   max_depth=2, gamma=0, min_child_weight=1, reg_lambda=2, colsample_bytree=1)

cross_val(xgb, cv, SMOTE())

### Random Forest

In [ ]:
rfc = RandomForestClassifier(n_estimators = 300, max_features=2, max_depth = 1, random_state=rd)

In [ ]:
cross_val(rfc, cv)

In [ ]:
rfc_balanced = RandomForestClassifier(n_estimators = 300, max_features=2, max_depth = 1, random_state=rd, class_weight='balanced')
cross_val(rfc_balanced, cv)

In [ ]:
cross_val(rfc, cv, resample=SMOTE())

In [ ]:
cross_val(rfc, cv, resample=TomekLinks(sampling_strategy='majority'))

In [ ]:
cross_val(rfc, cv, resample=NearMiss())

In [ ]:
# Plotting Important Features from RF
rfc = RandomForestClassifier(n_estimators = 300, max_features='sqrt', max_depth = 1, random_state=rd)
rfc.fit(X, y)
sort = rfc.feature_importances_.argsort()[-21: -1]
plt.barh([feature_names[s] for s in sort], rfc.feature_importances_[sort])
plt.xlabel("Feature Importance")

### Logistic Regression

In [ ]:
lr = LogisticRegression(penalty='l2', C=10, solver='liblinear')
cross_val(lr, cv)

In [ ]:
cross_val(lr, cv, SMOTE())

In [ ]:
# Getting importance coefficients from Lasso
lr.fit(X, y)
coefficients = lr.coef_
importance = list(np.abs(coefficients).reshape(-1))
posImportance = sorted([i for i in importance if i > 0])
for i in posImportance:
    print(feature_names[importance.index(i)])

### SVM

In [ ]:
svc = SVC(C=0.005, kernel='linear', probability=True)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=rd)
cross_val(svc, cv)